In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.decomposition import PCA, NMF
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib
import os

import matplotlib.pyplot as plt
plt.close('all')  # 全ての図を明示的に閉じる
matplotlib.use('Agg')  # その後バックエンドを切り替え

# グラフ出力用のディレクトリを作成
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# バックエンド設定
matplotlib.use('Agg')  # 非インタラクティブなバックエンドを使用

class AnalyzeIris:
    def __init__(self):
        self.data = None
        self.X = None
        self.y = None
        self.feature_names = None
        self.target_names = None
        self.model_scores = {}

    def get(self):
        """irisデータセットを取得し、必要な形式に変換する"""
        iris = load_iris()
        self.feature_names = iris.feature_names
        self.target_names = iris.target_names
        
        # データフレームに変換
        self.data = pd.DataFrame(data=iris.data, columns=self.feature_names)
        self.data['target'] = iris.target
        self.data['species'] = self.data['target'].map({
            0: self.target_names[0],
            1: self.target_names[1],
            2: self.target_names[2]
        })
        
        # X, yに分割
        self.X = self.data[self.feature_names]
        self.y = self.data['target']
        
        return self.data

    def get_correlation(self):
        """変数間の相関係数を計算して表示する"""
        corr = self.X.corr()
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr, annot=True, cmap='coolwarm')
        plt.title('Feature Correlation Matrix')
        plt.savefig(f'{output_dir}/correlation_matrix.png')
        plt.close()
        return corr

    def pair_plot(self, diag_kind=None):
        # データフレームの作成
        df = pd.DataFrame(data=self.X)
        df.columns = self.feature_names
        df['species'] = [self.target_names[i] for i in self.y]
    
        if diag_kind == 'hist' or diag_kind == 'kde':
            # 指定された対角成分のみ表示
            g = sns.pairplot(df, hue='species', diag_kind=diag_kind)
            g.fig.suptitle(f'Pair Plot with {diag_kind.upper()} on Diagonal', y=1.02)
            plt.savefig(f'{output_dir}/pairplot_{diag_kind}.png')
            plt.close()
        else:
            # ヒストグラムバージョン
            g1 = sns.pairplot(df, hue='species', diag_kind='hist')
            g1.fig.suptitle('Pair Plot with Histograms on Diagonal', y=1.02)
            plt.savefig(f'{output_dir}/pairplot_hist.png')
            plt.close()
        
            # KDEバージョン
            g2 = sns.pairplot(df, hue='species', diag_kind='kde')
            g2.fig.suptitle('Pair Plot with KDE on Diagonal', y=1.02)
            plt.savefig(f'{output_dir}/pairplot_kde.png')
            plt.close()
    
        return

    def all_supervised(self, n_neighbors=4):
        """複数の教師あり学習モデルを実行して評価する"""
        # 評価対象のモデル
        models = {
            'LogisticRegression': LogisticRegression(max_iter=1000),
            'LinearSVC': LinearSVC(max_iter=1000, dual='auto'), 
            'SVC': SVC(),
            'DecisionTreeClassifier': DecisionTreeClassifier(),
            'KNeighborsClassifier': KNeighborsClassifier(n_neighbors=n_neighbors),
            'LinearRegression': LinearRegression(),
            'RandomForestClassifier': RandomForestClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
            'MLPClassifier': MLPClassifier(max_iter=1000)
        }
        
        # K分割交差検証
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        
        # 結果格納用
        self.model_scores = {}
        
        # 各モデルに対して評価
        for name, model in models.items():
            print(f"=== {name} ===")
            test_scores = []
            train_scores = []
            
            for train_idx, test_idx in kf.split(self.X):
                X_train, X_test = self.X.iloc[train_idx], self.X.iloc[test_idx]
                y_train, y_test = self.y.iloc[train_idx], self.y.iloc[test_idx]
                
                model.fit(X_train, y_train)
                
                # トレーニングスコア計算
                if name == 'LinearRegression':
                    train_score = model.score(X_train, y_train)
                    test_score = model.score(X_test, y_test)
                else:
                    train_score = accuracy_score(y_train, model.predict(X_train))
                    test_score = accuracy_score(y_test, model.predict(X_test))
                
                train_scores.append(train_score)
                test_scores.append(test_score)
                
                print(f"test score: {test_score:.3f}, train score: {train_score:.3f}")
            
            # スコアを保存
            self.model_scores[name] = {
                'test_scores': test_scores,
                'train_scores': train_scores,
                'mean_test_score': np.mean(test_scores),
                'mean_train_score': np.mean(train_scores)
            }
            
            print()
        
        return self.model_scores

    def get_supervised(self):
        """学習結果をDataFrameで返す"""
        if not self.model_scores:
            return None
        
        results = []
        for model_name, scores in self.model_scores.items():
            for i, (test, train) in enumerate(zip(scores['test_scores'], scores['train_scores'])):
                results.append({
                    'model': model_name,
                    'fold': i+1,
                    'test_score': test,
                    'train_score': train
                })
        
        df_scores = pd.DataFrame(results)
        return df_scores

    def best_supervised(self):
        """最良のモデルを返す"""
        if not self.model_scores:
            return None, 0
        
        best_model = None
        best_score = 0
        
        for model_name, scores in self.model_scores.items():
            mean_score = scores['mean_test_score']
            if mean_score > best_score:
                best_score = mean_score
                best_model = model_name
        
        return best_model, best_score

    def plot_feature_importances_all(self):
        """特徴量の重要度を横棒グラフで表示する"""
        models_with_importances = [
            'DecisionTreeClassifier', 
            'RandomForestClassifier', 
            'GradientBoostingClassifier'
        ]

        # 表示するモデルの数をカウント
        available_models = [model for model in models_with_importances if model in self.model_scores]
        if not available_models:
            print("特徴量の重要度を持つモデルが見つかりません。先に all_supervised() を実行してください。")
            return None

        # 図を作成
        fig = plt.figure(figsize=(15, 10))

        # Irisデータセットの特徴量名をそのまま使用
        feature_names = self.feature_names

        for i, model_name in enumerate(models_with_importances):
            if model_name not in self.model_scores:
                continue
        
            # モデルを再学習
            if model_name == 'DecisionTreeClassifier':
                model = DecisionTreeClassifier()
            elif model_name == 'RandomForestClassifier':
                model = RandomForestClassifier()
            elif model_name == 'GradientBoostingClassifier':
                model = GradientBoostingClassifier()
    
            model.fit(self.X, self.y)
    
            # 重要度を取得
            importance = model.feature_importances_
    
            # インデックスでソート（降順）
            sorted_indices = np.argsort(importance)
    
            # サブプロット作成
            ax = fig.add_subplot(1, len(available_models), i+1)
    
            # 横棒グラフの作成
            y_pos = np.arange(len(feature_names))
            ax.barh(y_pos, importance[sorted_indices], align='center')
            ax.set_yticks(y_pos)
            ax.set_yticklabels([feature_names[idx] for idx in sorted_indices])
            ax.invert_yaxis()  # 上から下へ値が大きい順に表示
            ax.set_xlabel('Feature Importance (0-1.0)')
            ax.set_xlim(0, 1.0)  # X軸の範囲を0～1に設定
            ax.set_title(f'{model_name}')
    
        # 全モデルをまとめて1つの図として保存
        plt.tight_layout()  # レイアウトを調整
        plt.savefig(f'{output_dir}/feature_importance_all_models.png')
        plt.close()

    def visualize_decision_tree(self):
        """決定木を可視化する"""
        tree = DecisionTreeClassifier()
        tree.fit(self.X, self.y)
        
        plt.figure(figsize=(15, 10))
        plot_tree(tree, filled=True, feature_names=self.feature_names, class_names=self.target_names)
        plt.title('Decision Tree Visualization')
        plt.tight_layout()
        # ファイルに保存
        plt.savefig(f'{output_dir}/decision_tree.png')
        plt.close()
        return tree

    def plot_scaled_data(self):
        """異なるスケーリング手法でデータを変換し、LinearSVCの結果を評価する"""
        scalers = {
            'Original': None,
            'MinMaxScaler': MinMaxScaler(),
            'StandardScaler': StandardScaler(),
            'RobusScaler': RobustScaler(),
            'Normalizer': Normalizer()
        }
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        
        for train_idx, test_idx in kf.split(self.X):
            X_train, X_test = self.X.iloc[train_idx], self.X.iloc[test_idx]
            y_train, y_test = self.y.iloc[train_idx], self.y.iloc[test_idx]
            
            print("=" * 73)
            
            for name, scaler in scalers.items():
                if scaler:
                    X_train_scaled = scaler.fit_transform(X_train)
                    X_test_scaled = scaler.transform(X_test)
                else:
                    X_train_scaled = X_train
                    X_test_scaled = X_test
                
                model = LinearSVC(max_iter=1000, dual='auto')
                model.fit(X_train_scaled, y_train)
                
                train_score = accuracy_score(y_train, model.predict(X_train_scaled))
                test_score = accuracy_score(y_test, model.predict(X_test_scaled))
                
                print(f"{name:<15}: test score: {test_score:.3f}      train score: {train_score:.3f}     ")
        
        print("=" * 73)
        
        # 最初のスケーリングデータを返す (標準スケーリング)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(self.X)
        return pd.DataFrame(X_scaled, columns=self.feature_names)

    def plot_pca(self, n_components=2):
        """PCA分析を行い結果をプロットする"""
        # データのスケーリング
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(self.X)
        
        # PCA実行
        pca = PCA(n_components=n_components)
        X_pca = pca.fit_transform(X_scaled)
        
        # 結果をデータフレーム化
        df_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(n_components)])
        df_pca['species'] = self.data['species']
        
        # プロット
        plt.figure(figsize=(10, 8))
        sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='species', palette='viridis', s=100)
        plt.title('PCA of Iris Dataset')
        
        # 主成分の寄与率
        explained_variance = pca.explained_variance_ratio_
        plt.xlabel(f'PC1 ({explained_variance[0]:.2f})')
        plt.ylabel(f'PC2 ({explained_variance[1]:.2f})')
        
        plt.savefig(f'{output_dir}/pca_analysis.png')
        plt.close()
        
        return pd.DataFrame(X_scaled, columns=self.feature_names), df_pca, pca

    def plot_nmf(self, n_components=2):
        """NMF分析を行い結果をプロットする"""
        # データのスケーリング (負の値は使えないのでMinMaxScalerを使用)
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(self.X)
        
        # NMF実行 (反復回数を増やして警告を減らす)
        nmf = NMF(n_components=n_components, random_state=42, max_iter=400)
        X_nmf = nmf.fit_transform(X_scaled)
        
        # 結果をデータフレーム化
        df_nmf = pd.DataFrame(X_nmf, columns=[f'NMF{i+1}' for i in range(n_components)])
        df_nmf['species'] = self.data['species']
        
        # プロット
        plt.figure(figsize=(10, 8))
        sns.scatterplot(data=df_nmf, x='NMF1', y='NMF2', hue='species', palette='viridis', s=100)
        plt.title('NMF of Iris Dataset')
        plt.savefig(f'{output_dir}/nmf_analysis.png')
        plt.close()
        
        return pd.DataFrame(X_scaled, columns=self.feature_names), df_nmf, nmf

    def plot_tsne(self):
        """t-SNE分析を行い結果をプロットする"""
        # t-SNE実行 (スケールしていない元データを使用)
        tsne = TSNE(n_components=2, random_state=42)
        X_tsne = tsne.fit_transform(self.X)
        
        # 結果をデータフレーム化
        df_tsne = pd.DataFrame(X_tsne, columns=['t-SNE1', 't-SNE2'])
        df_tsne['species'] = self.data['species']
        
        # プロット
        plt.figure(figsize=(10, 8))
        sns.scatterplot(data=df_tsne, x='t-SNE1', y='t-SNE2', hue='species', palette='viridis', s=100)
        plt.title('t-SNE of Iris Dataset')
        plt.savefig(f'{output_dir}/tsne_analysis.png')
        plt.close()
        
        return df_tsne

    def plot_k_means(self):
        """K-means分析を行い結果をプロットする"""
        # K-means実行
        kmeans = KMeans(n_clusters=3, random_state=42)
        clusters = kmeans.fit_predict(self.X)
        
        # 結果をデータフレーム化
        df_kmeans = self.X.copy()
        df_kmeans['cluster'] = clusters
        df_kmeans['actual'] = self.y
        
        # プロット
        plt.figure(figsize=(15, 5))
        
        # K-means結果
        plt.subplot(1, 2, 1)
        sns.scatterplot(x=self.X.iloc[:, 0], y=self.X.iloc[:, 1], hue=clusters, palette='viridis', s=100)
        plt.title('K-means Clustering')
        plt.xlabel(self.feature_names[0])
        plt.ylabel(self.feature_names[1])
        
        # 実際のラベル
        plt.subplot(1, 2, 2)
        sns.scatterplot(x=self.X.iloc[:, 0], y=self.X.iloc[:, 1], hue=self.y, palette='viridis', s=100)
        plt.title('Actual Classes')
        plt.xlabel(self.feature_names[0])
        plt.ylabel(self.feature_names[1])
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/kmeans_analysis.png')
        plt.close()
        
        print("KMeans法で予測したラベル:")
        print(clusters)
        print("\n実際のラベル:")
        print(self.y.values)
        
        return df_kmeans

    def plot_dendrogram(self, truncate=False):
        """階層的クラスタリングのデンドログラムをプロットする"""
        try:
            # リンケージ行列を計算 (DataFrameをnumpy arrayに変換)
            X_array = self.X.values
            linked = linkage(X_array, 'ward')
            
            # デンドログラムをプロット
            plt.figure(figsize=(12, 8))
            dendrogram(
                linked,
                truncate_mode='lastp' if truncate else None,
                p=10 if truncate else None,
                leaf_font_size=10.,
                orientation='top'
            )
            plt.title('Hierarchical Clustering Dendrogram')
            plt.xlabel('Sample index')
            plt.ylabel('Distance')
            
            # ファイル名に truncated を追加するかどうか
            truncate_str = "_truncated" if truncate else ""
            plt.savefig(f'{output_dir}/dendrogram{truncate_str}.png')
            plt.close()
            
            return linked
        except Exception as e:
            print(f"デンドログラムの作成中にエラーが発生しました: {e}")
            return None

    def plot_dbscan(self, scaling=False, eps=0.5, min_samples=5):
        """DBSCAN分析を行い結果をプロットする"""
        # データのスケーリング (オプション)
        if scaling:
            scaler = StandardScaler()
            X_dbscan = scaler.fit_transform(self.X)
        else:
            X_dbscan = self.X.values
        
        # DBSCAN実行
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        clusters = dbscan.fit_predict(X_dbscan)
        
        # 結果をデータフレーム化
        df_dbscan = self.X.copy()
        df_dbscan['cluster'] = clusters
        
        try:
            # 特徴量の組み合わせをプロット
            fig, axes = plt.subplots(2, 3, figsize=(16, 10))
            
            # プロットのためのカラーマップ (-1はノイズ点で黒にする)
            cmap = plt.cm.viridis
            cmaplist = [cmap(i) for i in range(cmap.N)]
            cmaplist[0] = (0, 0, 0, 1.0)  # ノイズ点を黒に
            cmap_custom = plt.matplotlib.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            
            # 特徴量ペアのプロット
            feature_pairs = [
                (0, 1), (0, 2), (0, 3),
                (1, 2), (1, 3), (2, 3)
            ]
            
            for i, (f1, f2) in enumerate(feature_pairs):
                row, col = i // 3, i % 3
                axes[row, col].scatter(X_dbscan[:, f1], X_dbscan[:, f2], c=clusters, cmap=cmap_custom, s=50)
                axes[row, col].set_xlabel(self.feature_names[f1])
                axes[row, col].set_ylabel(self.feature_names[f2])
                axes[row, col].set_title(f'{self.feature_names[f1]} vs {self.feature_names[f2]}')
            
            plt.tight_layout()
            plt.suptitle('DBSCAN Clustering' + (' (Scaled)' if scaling else ''), y=1.02, fontsize=16)
            
            # スケーリングの有無をファイル名に反映
            scaled_str = "_scaled" if scaling else ""
            plt.savefig(f'{output_dir}/dbscan{scaled_str}.png')
            plt.close()
            
        except Exception as e:
            print(f"DBSCAN散布図の作成中にエラーが発生しました: {e}")
        
        print("Cluster Memberships:", clusters)
        
        return df_dbscan

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import matplotlib
import sys
from iris import AnalyzeIris

def print_task_header(task_number, task_name):
    """タスクのヘッダーを表示する"""
    print("\n" + "=" * 50)
    print(f"タスク {task_number}: {task_name}")
    print("=" * 50)

def task_1_load_data():
    """課題1: データの読み込み"""
    print_task_header(1, "Irisデータセットの読み込み")
    
    iris = AnalyzeIris()
    data = iris.get()
    print(f"データサイズ: {data.shape}")
    print("データサンプル:")
    print(data.head(20))
    
    return iris

def task_2_correlation(iris):
    """課題2: 相関分析"""
    print_task_header(2, "変数間の相関関係を確認")
    
    try:
        correlation = iris.get_correlation()
        print("相関行列:")
        print(correlation)
    except Exception as e:
        print(f"相関行列の表示中にエラーが発生しました: {e}")
    
    return correlation

def task_3_pair_plot(iris):
    """課題3: ペアプロット (ヒストグラム)"""
    print_task_header(3, "ペアプロット (対角線にヒストグラム)")
    
    try:
        df = iris.pair_plot()
        return df
    except Exception as e:
        print(f"ペアプロットの表示中にエラーが発生しました: {e}")
        return None

def task_4_pair_plot_kde(iris):
    """課題4: ペアプロット (KDE)"""
    print_task_header(4, "ペアプロット (対角線にカーネル密度推定)")
    
    try:
        df = iris.pair_plot(diag_kind="kde")
        return df
    except Exception as e:
        print(f"KDEペアプロットの表示中にエラーが発生しました: {e}")
        return None

def task_5_supervised_learning(iris):
    """課題5: 教師あり学習モデルの評価"""
    print_task_header(5, "複数の教師あり学習モデルを評価")
    
    print("9つのモデルをクロスバリデーションで評価します...")
    model_scores = iris.all_supervised(n_neighbors=4)
    return model_scores

def task_6_get_learning_results(iris):
    """課題6: 学習結果の取得"""
    print_task_header(6, "学習結果をデータフレームで取得")
    
    df_scores = iris.get_supervised()
    print(df_scores.head(15))  # 最初の数行を表示
    return df_scores

def task_7_summary_stats(df_scores):
    """課題7: 結果の要約統計量"""
    print_task_header(7, "スコアの要約統計量")
    
    if df_scores is not None:
        df_summary = df_scores.describe()
        print(df_summary)
        return df_summary
    else:
        print("スコアデータが見つかりません。先に課題5と6を実行してください。")
        return None

def task_8_best_model(iris):
    """課題8: 最良のモデルを特定"""
    print_task_header(8, "最良のモデルを特定")
    
    best_method, best_score = iris.best_supervised()
    print(f"BestMethod is {best_method} : {best_score:.4f}")
    return best_method, best_score

def task_9_feature_importance(iris):
    """課題9: 特徴量の重要度を可視化"""
    print_task_header(9, "特徴量の重要度をプロット")
    
    try:
        iris.plot_feature_importances_all()
    except Exception as e:
        print(f"特徴量重要度の表示中にエラーが発生しました: {e}")

def task_10_decision_tree(iris):
    """課題10: 決定木の可視化"""
    print_task_header(10, "決定木を可視化")
    
    try:
        tree = iris.visualize_decision_tree()
        return tree
    except Exception as e:
        print(f"決定木の表示中にエラーが発生しました: {e}")
        return None

def task_11_data_scaling(iris):
    """課題11: データスケーリングとその効果"""
    print_task_header(11, "異なるスケーリング手法の効果を確認")
    
    print("各スケーリング手法でLinearSVCを評価します...")
    train_data = iris.plot_scaled_data()
    return train_data

def task_12_pca(iris):
    """課題12: PCA分析"""
    print_task_header(12, "PCA (主成分分析)")
    
    try:
        X_scaled, df_pca, pca = iris.plot_pca(n_components=2)
        print("PCAの主成分:")
        print(pca.components_)
        return X_scaled, df_pca, pca
    except Exception as e:
        print(f"PCA分析の表示中にエラーが発生しました: {e}")
        return None, None, None

def task_13_nmf(iris):
    """課題13: NMF分析"""
    print_task_header(13, "NMF (非負値行列因子分解)")
    
    try:
        X_scaled, df_nmf, nmf = iris.plot_nmf(n_components=2)
        return X_scaled, df_nmf, nmf
    except Exception as e:
        print(f"NMF分析の表示中にエラーが発生しました: {e}")
        return None, None, None

def task_14_tsne(iris):
    """課題14: t-SNE分析"""
    print_task_header(14, "t-SNE分析")
    
    try:
        df_tsne = iris.plot_tsne()
        return df_tsne
    except Exception as e:
        print(f"t-SNE分析の表示中にエラーが発生しました: {e}")
        return None

def task_15_kmeans(iris):
    """課題15: K-means分析"""
    print_task_header(15, "K-means分析")
    
    try:
        df_kmeans = iris.plot_k_means()
        return df_kmeans
    except Exception as e:
        print(f"K-means分析の表示中にエラーが発生しました: {e}")
        return None

def task_16_dendrogram(iris):
    """課題16: 階層的クラスタリング (デンドログラム)"""
    print_task_header(16, "階層的クラスタリング (デンドログラム)")
    
    try:
        linked = iris.plot_dendrogram()
        return linked
    except Exception as e:
        print(f"デンドログラムの表示中にエラーが発生しました: {e}")
        return None

def task_17_truncated_dendrogram(iris):
    """課題17: 簡略化したデンドログラム"""
    print_task_header(17, "簡略化したデンドログラム")
    
    try:
        linked = iris.plot_dendrogram(truncate=True)
        return linked
    except Exception as e:
        print(f"簡略化デンドログラムの表示中にエラーが発生しました: {e}")
        return None

def task_18_dbscan(iris):
    """課題18: DBSCAN分析"""
    print_task_header(18, "DBSCAN分析")
    
    try:
        df_dbscan = iris.plot_dbscan()
        return df_dbscan
    except Exception as e:
        print(f"DBSCAN分析の表示中にエラーが発生しました: {e}")
        return None

def task_19_scaled_dbscan(iris):
    """課題19: スケーリングありのDBSCAN分析"""
    print_task_header(19, "スケーリングありのDBSCAN分析")
    
    try:
        df_dbscan_scaled = iris.plot_dbscan(scaling=True, eps=0.5, min_samples=5)
        return df_dbscan_scaled
    except Exception as e:
        print(f"スケーリングありDBSCAN分析の表示中にエラーが発生しました: {e}")
        return None

def task_20_clustering_comparison():
    """課題20: クラスタリング手法の比較"""
    print_task_header(20, "最終課題: クラスタリング手法の比較")
    
    comparison = """
    KMeans, 階層的クラスタリング, DBSCANの比較:
    
    1. K-means:
       - 長所: 単純で理解しやすい、計算効率が良い
       - 短所: クラスタ数を事前指定、球形クラスタのみ対応
       - Irisデータでは: k=3で比較的適切に機能するが、完全には分離できない
       
    2. 階層的クラスタリング:
       - 長所: クラスタ数事前指定不要、階層関係を視覚化
       - 短所: 計算コスト高、大規模データに不向き
       - Irisデータでは: 種の分類学的関係を明確に示す
       
    3. DBSCAN:
       - 長所: クラスタ数事前指定不要、異常値検出、任意形状対応
       - 短所: パラメータ設定が難しい、密度差のあるクラスタに弱い
       - Irisデータでは: パラメータによって結果が大きく変わる
       
    結論: Irisデータセットでは種の数が既知なのでK-meansが単純かつ効果的ですが、
    実際の応用では事前知識の有無によって最適手法が異なります。
    """
    
    print(comparison)
    return comparison

def print_menu():
    """メニューを表示する"""
    print("\n" + "=" * 50)
    print("Irisデータセット分析メニュー")
    print("=" * 50)
    print("1.  データの読み込み")
    print("2.  相関分析")
    print("3.  ペアプロット (ヒストグラム)")
    print("4.  ペアプロット (KDE)")
    print("5.  教師あり学習モデルの評価")
    print("6.  学習結果の取得")
    print("7.  結果の要約統計量")
    print("8.  最良のモデルを特定")
    print("9.  特徴量の重要度を可視化")
    print("10. 決定木の可視化")
    print("11. データスケーリング効果を確認")
    print("12. PCA分析")
    print("13. NMF分析")
    print("14. t-SNE分析")
    print("15. K-means分析")
    print("16. 階層的クラスタリング")
    print("17. 簡略化したデンドログラム")
    print("18. DBSCAN分析")
    print("19. スケーリングありのDBSCAN分析")
    print("20. クラスタリング手法の比較")
    print("0.  終了")
    print("=" * 50)

def main():
    """メインプログラム"""
    print("===== Irisデータセット分析スクリプト =====")
    print(f"Matplotlibのバックエンド: {matplotlib.get_backend()}")
    
    iris = None
    df_scores = None
    
    while True:
        print_menu()
        try:
            choice = int(input("実行する課題番号を入力してください (0-20): "))
        except ValueError:
            print("数字を入力してください")
            continue
        
        if choice == 0:
            print("プログラムを終了します")
            break
        
        if choice == 1:
            iris = task_1_load_data()
        elif choice >= 2 and choice <= 20:
            if iris is None and choice <= 19:
                print("先に課題1を実行してデータをロードしてください")
                continue
                
            if choice == 2:
                task_2_correlation(iris)
            elif choice == 3:
                task_3_pair_plot(iris)
            elif choice == 4:
                task_4_pair_plot_kde(iris)
            elif choice == 5:
                task_5_supervised_learning(iris)
            elif choice == 6:
                df_scores = task_6_get_learning_results(iris)
            elif choice == 7:
                task_7_summary_stats(df_scores)
            elif choice == 8:
                task_8_best_model(iris)
            elif choice == 9:
                task_9_feature_importance(iris)
            elif choice == 10:
                task_10_decision_tree(iris)
            elif choice == 11:
                task_11_data_scaling(iris)
            elif choice == 12:
                task_12_pca(iris)
            elif choice == 13:
                task_13_nmf(iris)
            elif choice == 14:
                task_14_tsne(iris)
            elif choice == 15:
                task_15_kmeans(iris)
            elif choice == 16:
                task_16_dendrogram(iris)
            elif choice == 17:
                task_17_truncated_dendrogram(iris)
            elif choice == 18:
                task_18_dbscan(iris)
            elif choice == 19:
                task_19_scaled_dbscan(iris)
            elif choice == 20:
                task_20_clustering_comparison()
        else:
            print("1から20までの数字を入力してください")
    
    print("\n===== 分析終了 =====")

if __name__ == "__main__":
    main()

===== Irisデータセット分析スクリプト =====
Matplotlibのバックエンド: Agg

Irisデータセット分析メニュー
1.  データの読み込み
2.  相関分析
3.  ペアプロット (ヒストグラム)
4.  ペアプロット (KDE)
5.  教師あり学習モデルの評価
6.  学習結果の取得
7.  結果の要約統計量
8.  最良のモデルを特定
9.  特徴量の重要度を可視化
10. 決定木の可視化
11. データスケーリング効果を確認
12. PCA分析
13. NMF分析
14. t-SNE分析
15. K-means分析
16. 階層的クラスタリング
17. 簡略化したデンドログラム
18. DBSCAN分析
19. スケーリングありのDBSCAN分析
20. クラスタリング手法の比較
0.  終了

タスク 1: Irisデータセットの読み込み
データサイズ: (150, 6)
データサンプル:
    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                 5.1               3.5                1.4               0.2   
1                 4.9               3.0                1.4               0.2   
2                 4.7               3.2                1.3               0.2   
3                 4.6               3.1                1.5               0.2   
4                 5.0               3.6                1.4               0.2   
5                 5.4               3.9                1.7               0.4   
6                 4.